Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Stop warnings

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

Load the training and testing dataset

In [ ]:
dataset_path = 'dataset'
train = pd.read_csv(os.path.join(dataset_path, 'sign_mnist_train.csv'))
test = pd.read_csv(os.path.join(dataset_path, 'sign_mnist_test.csv'))
labels = train['label'].values # Extract labels

Remove the label column and binarize the labels

In [ ]:
train.drop('label', axis = 1, inplace = True)
label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(labels)

Convert the data into 2d np array because it's the format Tensorflow uses

In [ ]:
images = train.values
images = np.array([np.reshape(i, (28, 28, 1)) for i in images])

Split the data into test and train

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train, labels, test_size=0.3, random_state=420)

Define Hyperparameters

In [ ]:
batch_size = 128
num_classes = 24
epochs = 10

Normalize the data to be between 0 and 1 and reshape them to be compatible with Tensorflow

In [ ]:
x_train = x_train.values / 255
x_test = x_test.values / 255
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

Print a random image and its label

In [ ]:
index = np.random.randint(0, len(images))
print(labels[index])
plt.imshow(images[index], cmap='gray', vmin=0, vmax=255)

Initalize the model

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.20))

model.add(Dense(num_classes, activation='softmax'))

Compile the model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer= Adam(),
              metrics=['accuracy'])

Train the model

In [ ]:
history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=epochs, batch_size = batch_size)

Showing the increase in accuracy as number of epochs increases

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train', 'test'])

plt.show()

Test the model on unseen data

In [ ]:
test_labels = test['label']
test.drop('label', axis=1, inplace=True)
test_labels = label_binarizer.fit_transform(test_labels)

test_images = test.values
test_images = np.array([np.reshape(i, (28, 28, 1)) for i in test_images])
test_images = test_images / 255.0

y_pred = model.predict(test_images)

Check accuracy on unseen data

In [ ]:
accuracy_score(test_labels, y_pred.round())